# **Bài tập lớn cuối kỳ**
- Mssv: 1712601
- Họ và tên: Trịnh Văn Minh

# Ý tưởng để tái tạo ảnh mật



- Nhân ma trận nghịch đảo để giải hệ phương trình 
- Từ đó tìm ra các pixel $P_i$
- Hệ có dạng

\begin{equation*}
\begin{bmatrix}
P_1x_1^0 & P_2x_1^2 & P_3x_1^2 \\
P_1x_2^0 & P_2x_2^2 & P_3x_2^2 \\
P_1x_3^0 & P_2x_3^2 & P_3x_3^2
\end{bmatrix}
=
\begin{bmatrix}
y_1 \\
y_2 \\
y_3
\end{bmatrix}
\end{equation*}

- Tương đương

\begin{equation*}
\begin{bmatrix}
x_1^0 & x_1^2 & x_1^2 \\
x_2^0 & x_2^2 & x_2^2 \\
x_3^0 & x_3^2 & x_3^2
\end{bmatrix}
\begin{bmatrix}
P_1 \\
P_2 \\
P_3
\end{bmatrix}
=
\begin{bmatrix}
y_1 \\
y_2 \\
y_3
\end{bmatrix}
\end{equation*}

- Áp dụng phương pháp nhân nghịch đảo để tính các pixel ($P_i$)

\begin{equation*}
A 
= 
\begin{bmatrix}
x_1^0 & x_1^2 & x_1^2 \\
x_2^0 & x_2^2 & x_2^2 \\
x_3^0 & x_3^2 & x_3^2
\end{bmatrix}
\end{equation*}

\begin{equation*}
X 
= 
\begin{bmatrix}
P_1 \\
P_2 \\
P_3
\end{bmatrix}
\end{equation*}

\begin{equation*}
B 
= 
\begin{bmatrix}
y_1 \\
y_2 \\
y_3
\end{bmatrix}
\end{equation*}

$A.X = B $

$X^T.A^T$ = $B^T$

$X^T$ = $B^T.(A^T)^(-1)$

# Phần code


In [1]:
import math
from PIL import Image
import numpy as np
from sympy import Matrix

In [2]:
def compute_polynomial(p, coefs, xs):
    '''
    Tính giá trị của hàm đa thức.

    Các tham số:
        coefs (mảng numpy một chiều): Các hệ số của hàm đa thức, 
                                        f(x) = coefs[0] + coefs[1]*x + coefs[2]*x^2 + ...
        xs (mảng numpy một chiều): Các giá trị x mà tại đó cần tính f(x).
    Giá trị trả về:
        Mảng numpy một chiều: Phần tử chỉ số i trong mảng này bằng f(xs[i]).
    '''
    # TODO
    f_xs = np.zeros(xs.shape, dtype=int)
    for i in range(len(coefs)):
        xs_pow_i = np.ones(xs.shape, dtype=int)
        for j in range(i):
            xs_pow_i = (xs_pow_i * xs) % p

        f_xs = (f_xs + coefs[i] * xs_pow_i % p) % p

    return f_xs

def convert_pixels_to_mode(pixels, mode=0):
  count = 0
  # lossy
  if mode == 0:
      for r in range(pixels.shape[0]):
          for c in range(pixels.shape[1]):
              if pixels[r,c] > 250:
                  pixels[r,c] = 250
      return pixels
  # mode 1 or any: lossless
  elif mode == 1:
      pixels = pixels.flatten()
      new_pixels = np.array([], dtype=np.uint8)

      for i in range(pixels.size):
        if pixels[i] >= 250:
          count+=1
          new_pixels = np.append(new_pixels, [250, pixels[i] - 250])
        else:
          new_pixels = np.append(new_pixels, pixels[i])
      return new_pixels


def split(ori_image, split_image, n, k, p=251, mode=0):
    '''
    Hàm phân chia ảnh mật ori_image m pixel thành k phần (k nhóm) sao cho chỉ khi có ít nhất là k phần (k <= n)
    thì mới tái tạo được ori_image, còn ít hơn k phần thì sẽ không biết gì về s.

    Các tham số:
        ori_image (*.bmp, *.png): Ảnh cần chia sẻ.
        split_image (*.bmp, *.png): Phần tên không bao gồm số thứ tự (vd part.bmp thay vì part0.bmp)
        n (int): Số phần cần phân chia.
        k (int): Ngưỡng k (k <= n).
        mode: 0 (lossy)
              1 (lossless)
    Giá trị trả về:
        n ảnh mang tên split_image$.bmp / split_image$.png
        xs mảng x
    '''
    pixels = np.array(Image.open(ori_image))

    ori_shape = pixels.shape
    m = pixels.size

    # convert pixels to mode
    pixels = convert_pixels_to_mode(pixels, mode)
    new_m = pixels.size

    # padding 0 -> m % k == 0
    padded_pixels = np.append(pixels, (k - new_m % k)*[0])
    
    # reshape into group
    padded_pixels = padded_pixels.reshape(-1, k)
    groups_len = padded_pixels.shape[0]
    # 2D array
    result_arr = np.zeros((groups_len, n), dtype=np.uint8)

    for i in range(groups_len):
        # Lấy các hệ số từ k pixels
        coefs = padded_pixels[i]
        xs = np.arange(n) + 1
        ys = compute_polynomial(p, coefs, xs)
        ys = ys.astype(np.uint8)

        result_arr[i] = ys

    # Tách từng phần, tách từng ảnh
    for i in range(n):
        new_pixels = result_arr[:, i]
        # padding
        new_pixels = np.append(new_pixels, [0]*(m - len(new_pixels)))
        new_pixels = new_pixels.reshape(ori_shape[0], ori_shape[1])
        new_pixels= new_pixels.astype(np.uint8)
        
        # Ghi new pixels xuống file
        Image.fromarray(new_pixels).save(split_image.split('.')[0] + str(i + 1) + '.' + split_image.split('.')[1])
    return xs, new_m

## Hàm get_ys_group 
- Với tham số đầu vào split_image (vd: part.bmp thay vì part1.bmp)
- Hàm này sẽ trả về các nhóm các ys, kích thước (512*512), shape(512,512)

In [3]:
def get_ys_group(split_image, new_pixels_size, n, k, p=251):
    pixels_part = np.array(Image.open(split_image.split('.')[0] + '1.' + split_image.split('.')[1]))
    old_pixels_size = pixels_part.size
    shape = pixels_part.shape
    # kích thước ban đầu của group (chưa padding)
    ori_size = math.ceil(new_pixels_size / k)
    
    ys_group = np.zeros((n, ori_size), dtype=int)
    for i in range(n):
        pixels_part = np.array(Image.open(
            split_image.split('.')[0] + str(i + 1) + '.' + split_image.split('.')[1]))
        # loại bỏ padding
        pixels_part = pixels_part.flatten()[:ori_size]
        ys_group[i] = pixels_part

    ys_group = ys_group.T
    return ys_group, old_pixels_size, shape

## Hàm get_A 
- Nhận vào tham số `new_xs` là một mảng bao gồm các xs đã được hoán vị ngẫu nhiên
- Tạo ra mảng `xs_pow_i_arr` là một matrix gôm các giá trị
\begin{equation*}
A 
= 
\begin{bmatrix}
x_1^0 & x_1^2 & x_1^2 \\
x_2^0 & x_2^2 & x_2^2 \\
x_3^0 & x_3^2 & x_3^2
\end{bmatrix}
\end{equation*}

- Trà về một matrix A đã nghịch đảo và mod 251



In [4]:
def get_A(new_xs, k, p=251):
    # print(new_xs)
    xs_pow_i_arr = np.zeros((k, k), dtype=np.uint8)
    for i in range(len(new_xs)):
        xs_pow_i = np.ones(new_xs.shape, dtype=np.uint8)
        for j in range(i):
            xs_pow_i = (xs_pow_i * new_xs) % 251
        xs_pow_i_arr[i] = xs_pow_i
    # print(xs_pow_i_arr)

    # inverse matrix mod p
    A = Matrix(xs_pow_i_arr)  # keyMatrix is your basic matrix ndrarray format
    A = A.inv_mod(p)  # 251
    return A

## Hàm join
- Nhận vào các tham số đã được cắt ra bởi các index ngẫu nhiên của xs
- Trả về kết quả là một mảng gồm các pixel được tính bằng

 `pixels_part = matrix_B * matrix_A % p`

In [5]:
def join(new_xs, new_ys_group, k, p=251, mode=0):
    '''
    Tái tạo tin mật s từ n' phần của tin mật (n' >= k).

    Các tham số:
        xs, ys (2 mảng numpy một chiều, len = n'): n' phần của tin mật: phần thứ 1 là (xs[0], ys[0]), 
                                                                        phần thứ 2 là (xs[1], ys[1]), 
                                                                        ...
        k (int): Ngưỡng k mà đã dùng khi phân chia tin mật.
    Giá trị trả về:
        int: Tin mật s được tái tạo,
                trong trường hợp không đủ số phần để tái tạo tin mật thì trả về None.
    '''
    # TODO
    if len(new_xs) < k:
        return None

    # A inversed, mod 251
    matrix_A = get_A(new_xs, k, p)


    pixels = np.array([], dtype=int)
    for i in range(new_ys_group.shape[0]):
        matrix_B = Matrix(new_ys_group[i]).reshape(1, k)  # 1, k
        pixels_part = matrix_B * matrix_A % p
        pixels = np.append(pixels, pixels_part)


    # lossless
    if mode == 1:
      pixels_part_ori = np.array([], dtype=np.uint8) # lossless
      j = 0
      while (j < len(pixels)):
        if (pixels[j] == 250):
          pixels_part_ori = np.append(pixels_part_ori, pixels[j] + pixels[j + 1])
          j += 1
        else:
          pixels_part_ori = np.append(pixels_part_ori, pixels[j])  
        j+=1
      
      return pixels_part_ori.astype(np.uint8)

    # lossy
    return pixels.astype(np.uint8)

# Phân chia ảnh mật: chia nhóm, lấy kích thước ban đầu, shape

In [6]:
# Phân chia ảnh mật
p = 251
k = 3
n = 5

# mode = 0
mode = 1

split_image = 'parrot_part.png'
ori_image = 'cover2.png'


# split_image = 'girl_part.bmp'
# ori_image = 'cover.bmp'


# mode: 0 (lossy), 1(lossless)
xs, new_pixels_size = split(ori_image, split_image, n, k, p, mode)

# chia nhóm
ys_group, size, shape = get_ys_group(split_image, new_pixels_size, n, k, p)
print(ys_group)
print(xs)
print(size, shape)

[[  0   0   0   0   0]
 [  1   1   1   1   1]
 [  2   6  12  20  30]
 ...
 [145  93 141  38  35]
 [110 229 153 133 169]
 [ 34  34  34  34  34]]
[1 2 3 4 5]
30000 (200, 150)


# Chọn ngẫu nhiên n' phần để tái tạo tin mật 
- cover2.png (kích thước nhỏ chạy nhanh)
- Mất khoảng 4 phút (cover.bmp, kích thước lớn) với n = 5, k = 3, mode = 0 (lossy)

In [7]:
# -------------------------------------
#   mất khoảng 4 phút để hoàn tất với ảnh cover.bmp vì kích thước lớn
#--------------------------------------

rand_idxs = np.random.permutation(np.arange(n))


# -------------------------------------
n_prime = k - 1

new_xs = xs[rand_idxs[:n_prime]]
new_ys_group = ys_group[:, rand_idxs[:n_prime]]

rec_s = join(new_xs, new_ys_group, k, p)
print(rec_s)

#--------------------------------------

n_prime = k

new_xs = xs[rand_idxs[:n_prime]]
new_ys_group = ys_group[:, rand_idxs[:n_prime]]

rec_s = join(new_xs, new_ys_group, k, p, mode)
rec_s = rec_s[:size].reshape(shape)
print(rec_s)

# Ghi new pixels xuống file
Image.fromarray(rec_s).save('reconstructor_cover2.png')

None
[[  0   0   0 ...   3   0   0]
 [  0   0   0 ...  33  13   0]
 [  4   1   0 ...  96  65   6]
 ...
 [124 133 139 ...  33  13   8]
 [128 138 145 ...  35  28  28]
 [135 137 136 ...  35  28  34]]


# Kiểm tra lại kết quả

In [8]:

pixels = np.array(Image.open(ori_image))
print(pixels)
print(pixels.size)
print(pixels.shape)
print(pixels.dtype)

print(rec_s)
print(rec_s.size)
print(rec_s.shape)
print(rec_s.dtype)

number_of_equal_elements = np.sum(pixels==rec_s)
total_elements = np.multiply(*pixels.shape)
percentage = number_of_equal_elements/total_elements

print('total number of elements: \t\t{}'.format(total_elements))
print('number of identical elements: \t\t{}'.format(number_of_equal_elements))
print('number of different elements: \t\t{}'.format(total_elements-number_of_equal_elements))
print('percentage of identical elements: \t{:.2f}%'.format(percentage*100))

[[  0   0   0 ...   3   0   0]
 [  0   0   0 ...  33  13   0]
 [  4   1   0 ...  96  65   6]
 ...
 [124 133 139 ...  33  13   8]
 [128 138 145 ...  35  28  28]
 [135 137 136 ...  35  28  34]]
30000
(200, 150)
uint8
[[  0   0   0 ...   3   0   0]
 [  0   0   0 ...  33  13   0]
 [  4   1   0 ...  96  65   6]
 ...
 [124 133 139 ...  33  13   8]
 [128 138 145 ...  35  28  28]
 [135 137 136 ...  35  28  34]]
30000
(200, 150)
uint8
total number of elements: 		30000
number of identical elements: 		30000
number of different elements: 		0
percentage of identical elements: 	100.00%


# Báo Cáo


# Mô tả thuật toán đã cài đặt (Thien & Lin)

- Thuật toán shamir1 và shamir2 cặt đặt lại từ code demo của thầy
- Còn với Thien & lien cải tiến từ shamir2 
- Giải thích code chi tiết trong từng khối code

# Vấn đề của thuật toán shamir với trường số nguyên

Vấn đề tràn số:
- Thuật toán shamir là có dạng phương trình **bậc k**
- Các giá trị k, n, a lớn dẫn đến tràn số vì kiểu dữ liệu int trong các ngôn ngữ lập trình là cố giá trị giới hạn.

Vấn đề bảo mật:
- Phải có ít nhất k phần của Secret mới có thể tính ra Secret
- Đối với trường số nguyên vô hạn thì chỉ cần sở hữu k' ít hơn k của tin mật thì ta có thể:
    - Tái tạo lại các điểm k'
    - Tái tạo lại hàm số, đường cong dựa vào brute-fore, vết cạn (thế các giá trị x vào để tìm thêm thông tin)
    - k' càng gần bằng k thì việc tìm ngược lại hàm ban đầu càng dễ
    - Biểu diễn các điểm trên trục toạ độ ta có thể đoán trước được điểm tiếp theo
    - vd: k = 3 thì ta sẽ có đa thức bậc k - 1. Dựa vào hai điểm ta có thể đoán trước điẻm còn lại
  
- Dẫn đến việc khó bảo mật thông tin



# Tại sao trường hữu hạn có thể giải quyết được vấn đề này ?
Vấn đề 1: tràn số
- Đối với trường hữu hạn thì các giá trị bị giới hạn trong 1 vùng (modulo p)
- Giá trị P là số nguyên tố nhỏ hơn giá trị mà gây ra sự tràn số.
- Do module cho p nên đảm bảo được vấn đề tràn số không xảy ra.

Vấn đề 2: Bảo mật
- Trong trường hữu hạn thì khi tất cả các điểm được biểu diễn trên trục toạ độ. Ta có thể thấy các điểm xuất hiện gần như ngẫu nhiên, rất khó đễ dự đoán được hình dạng ban đầu của hàm số.
- Dẫn đến bảo mật tốn hơn, tránh được việc vét cạn (brute-fore)
 